# 기본 세팅

In [1]:
# 모듈 import
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
import pandas as pd
from urllib.parse import urlparse, parse_qs, urlencode, urlunparse

# 페이지 맨 아래까지 스크롤 다운하는 함수
def scroll_down():
    # 끝까지 스크롤 다운
    last_height = driver.execute_script("return document.body.scrollHeight")

    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")

        # 페이지 로드 대기
        time.sleep(2)  # 페이지 로딩 대기 시간 조정 가능

        # 새로운 높이 계산
        new_height = driver.execute_script("return document.body.scrollHeight")

        # 더 이상 스크롤할 내용이 없으면 종료
        if new_height == last_height:
            break
        last_height = new_height

# 텍스트 찾기 함수
def find_all_text(x):
    if x:
        x_texts = [i.text.strip() for i in x]
        x_text = ', '.join(x_texts)
    else:
        x_text = "Not found"
    
    return x_text

# 메뉴 클릭 함수
def click_menu(menu):
    # 탭 클릭
    try:
    # CSS 선택자를 사용하여 'veBoZ' 클래스를 가진 <span> 요소 중 텍스트가 menu인 요소 찾기
        menu_buttons = driver.find_elements(By.CSS_SELECTOR, 'a._tab-menu span.veBoZ')

        # '메뉴' 텍스트를 가진 요소 클릭
        for button in menu_buttons:
            if menu in button.text:
                button.click()
                break
    except Exception as e:
        print(f"오류 발생: {e}")
        
def scroll_up():
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(2)  # 스크롤 후 페이지 로딩 대기
    
def add_query_param_to_url(url, param_name, param_value):
    """현재 URL에 쿼리 파라미터를 추가하여 반환"""
    url_parts = list(urlparse(url))
    query = dict(parse_qs(url_parts[4]))
    query[param_name] = param_value
    url_parts[4] = urlencode(query, doseq=True)
    return urlunparse(url_parts)

# 식당 홈 탭 크롤링
- 식당 이름
- 메뉴 카테고리
- 주소
- 전화번호
- 웹사이트 주소
- 영업 시간
- 역에서부터의 거리
- 서비스 목록
- 총 리뷰 수

In [2]:
def home_page_data():
    time.sleep(5)
    
    # 영업시간 더보기 버튼 클릭
    more_busshour_button = driver.find_elements(By.XPATH, '/html/body/div[3]/div/div/div/div[5]/div/div[2]/div[1]/div/div[2]/div/a/div/div')
    if more_busshour_button:
        more_busshour_button[0].click()
        time.sleep(2)  # 클릭 후 로딩 대기
    else:
        pass  # 버튼이 없으면 넘어감
    
    scroll_down()

    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # 데이터 쌓기
    # 가게 이름
    try:
        restaurant_name = soup.find('span', class_='GHAhO').text.strip()
    except:
        restaurant_name = float('nan')
        
    # 업종
    try:
        category = soup.find('span', class_='lnJFt').text.strip()
    except:
        category = float('nan')
        
    # 총 방문자 리뷰 수
    try:
        count_reviews = soup.find_all('em', class_='place_section_count')
        second_count_review = count_reviews[1].text.strip()
    except:
        second_count_review = float('nan')
        
    # 주소
    try:
        address = soup.find('div', class_='O8qbU tQY7D').find('span', class_='LDgIH').text.strip()
    except:
        address = float('nan')
       
    # 역 기준 거리
    try:
        distance = soup.find('div', class_='nZapA').text.strip()
    except:
        distance = float('nan')

    # 영업 시간
    try:
        business_hours = soup.find('a', class_='gKP9i RMgN0').get_text(separator=' ').strip()
    except:
        business_hours = float('nan')
        

    # 전화번호
    try:
        phone_number = soup.find('div', class_='O8qbU nbXkr').find('span', class_='xlx7Q').text.strip()
    except:
        phone_number = float('nan')
        
    # 홈페이지
    try:
        # 'div' 태그에서 class가 'O8qbU yIPfO'인 요소를 찾고, 그 안에 있는 'a' 태그를 모두 찾기
        website_links = soup.find('div', class_='O8qbU yIPfO').find_all('a', class_='place_bluelink')
        # website_links_hrefs = [link.get('href') for link in website_links]
    except:
        website_links = float('nan')
        
    # 부가 서비스 목록
    try:
        home_service = soup.find('div', class_='xPvPE').text.strip()
    except:
        home_service = float('nan')

    return restaurant_name, category, second_count_review, address, distance, business_hours, phone_number, website_links, home_service

# 메뉴 탭 크롤링
- 메뉴 텍스트 정보

In [3]:
def menu_page_data():
    #메뉴 탭 클릭
    click_menu('메뉴')
    
    time.sleep(5)
    scroll_down()
    
    # 메뉴 더보기 버튼 클릭
    while True:
        try:
            more_menu_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[6]/div/div[1]/div[2]/div/a')
            if more_menu_button.is_displayed() and more_menu_button.is_enabled():
                more_menu_button.click()
                time.sleep(2)  # 클릭 후 로딩 대기
            else:
                break
        except NoSuchElementException:
            break
    
    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(page_source, 'html.parser')

    # 메뉴 텍스트 내용 긁어오기
    try:
        menu_texts = soup.find_all('div', class_='MXkFw')
        if not menu_texts:  # 만약 찾은 결과가 비어있다면
            raise ValueError("First selector not found")
        menu_contents = [menu_text.get_text(separator='//').strip() for menu_text in menu_texts]
    except (AttributeError, ValueError):
        try:
            menu_texts = soup.find_all('div', class_='info_detail')
            if not menu_texts:  # 만약 찾은 결과가 비어있다면
                raise ValueError("Second selector not found")
            menu_contents = [menu_text.get_text(separator='//').strip() for menu_text in menu_texts]
        except (AttributeError, ValueError):
            menu_contents = float('nan')

#     # 메뉴판 이미지
#     menu_image_element = soup.find('div', class_='WKvXd').find('img')
#     if menu_image_element is not None:
#         menu_image = menu_image_element['src']
#     else:
#         menu_image = float('nan')

    try:
        menu_image_elements = soup.find_all('div', class_='WKvXd')
        menu_images = []
        for element in menu_image_elements:
            img = element.find('img')
            if img is not None:
                menu_images.append(img['src'])
            else:
                menu_images.append(float('nan'))
    except (AttributeError, TypeError, KeyError):
        menu_images = [float('nan')]
        
    # 결과 반환
    return menu_contents, menu_images

# 정보 탭 크롤링
- 소개글
- 편의시설 및 서비스
- 주차
- 좌석, 공간

In [4]:
import re

def info_page_data(ID):
#     # 정보 탭 클릭
#     click_menu('정보')

    # 정보 링크로 이동
    driver.get(f'https://m.place.naver.com/restaurant/{ID}/information')

    time.sleep(5)
    scroll_down()

    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    
    # 데이터 쌓기
    # 소개글
    #     try:
    #         introduce_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[5]/div/div[1]/div/div/div[2]/a')
    #         introduce_button.click()
    #         introduce = soup.find('div', class_='T8RFa').text.strip()
    #     except:
    #         introduce = float('nan')
    
    # 편의 시설 및 서비스
    try:
        service = soup.find_all('div', class_='owG4q')
        services = find_all_text(service)
        
    except:
        services = float('nan')
    
    # 주차
    try:
        parking = soup.find('div', class_='TZ6eS').text.strip()
    except:
        parking = float('nan')
    
    # 좌석, 공간
    try:
        seat = soup.find_all('li', class_='Lw5L1')
        seats = find_all_text(seat)
        
    except:
        seats = float('nan')
        
    # 소개
    try:
        info = soup.find('div', class_=re.compile(r'^T8RFa')).text
        
    except:
        info = float('nan')
    
    return services, parking, seats, info

# 리뷰 탭 크롤링
- 리뷰 텍스트 내용

In [5]:
def review_page_data():
    # 리뷰 탭 클릭
    click_menu('리뷰')
    time.sleep(5)
    
    # scroll_up()
    current_url = driver.current_url
    updated_url = add_query_param_to_url(current_url, 'reviewSort', 'recent')
    driver.get(updated_url)
    time.sleep(5)  # 페이지 로딩 대기
    
    # 키워드 리뷰 데이터 쌓기
    while True:
        try:
            # "더보기" 버튼의 XPath
            more_review_keyword_xpath = '/html/body/div[3]/div/div/div/div[6]/div[2]/div[1]/div/div/div[2]/a[1]'

            # "더보기" 버튼 요소 찾기
            more_review_keyword = driver.find_element(By.XPATH, more_review_keyword_xpath)

            # "더보기" 버튼 클릭
            more_review_keyword.click()

            # 클릭 후 페이지 로딩 대기
            time.sleep(2)

            # 'a' 태그의 'dP0sq' 클래스를 가진 요소 찾기
            dP0sq_elements = driver.find_elements(By.CLASS_NAME, 'dP0sq')

            if not dP0sq_elements:
                # 'dP0sq' 클래스를 가진 요소가 없으면 루프 종료
                break

        except NoSuchElementException:
            # "더보기" 버튼을 찾을 수 없으면 루프 종료
            break

    scroll_down()

    # 리뷰 키워드 정보 추출
    # 페이지 소스 가져오기
    page_source = driver.page_source

    # BeautifulSoup 객체 생성
    soup = BeautifulSoup(page_source, 'html.parser')

    # 모든 't3JSf' 태그와 'CUoLy' 태그를 찾음
    review_keywords_elements = soup.find_all('span', class_='t3JSf')
    review_keywords_count_elements = soup.find_all('span', class_='CUoLy')

    # 두 리스트의 길이를 확인하여 최소 길이만큼 순회
    min_length = min(len(review_keywords_elements), len(review_keywords_count_elements))

    # 키워드 데이터 리스트 생성
    review_keywords_data = []

    # 순서대로 함께 추가
    for i in range(min_length):
        review_keywords = review_keywords_elements[i].text.strip()
        review_keywords_count = review_keywords_count_elements[i].text.strip()
        review_keywords_data.append({
            'keyword': review_keywords,
            'count': review_keywords_count
        })
    
    scroll_down()
    
    # 끝까지 스크롤 다운 및 더보기 클릭 반복
    i = 0
    while i < 9:
        # 끝까지 스크롤 다운
        last_height = driver.execute_script("return document.body.scrollHeight")

        max_scroll_attempts = 1  # 최대 스크롤 시도 횟수 설정
        scroll_attempts = 0

        while True:
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # 페이지 로딩 대기 시간 조정 가능
            new_height = driver.execute_script("return document.body.scrollHeight")
            if new_height == last_height:
                scroll_attempts += 1
                if scroll_attempts >= max_scroll_attempts:
                    break
            else:
                scroll_attempts = 0  # 높이가 변경되면 시도 횟수 초기화
            last_height = new_height
        
        # 더보기 버튼 클릭
        try:
            more_review_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a')
            # 더보기 버튼이 있는 곳으로 스크롤
            driver.execute_script("arguments[0].scrollIntoView(true);", more_review_button)
            time.sleep(1)  # 스크롤 후 로딩 대기
            if more_review_button.is_displayed() and more_review_button.is_enabled():
                more_review_button.click()
                i += 1
                time.sleep(2)  # 클릭 후 로딩 대기
                
        except ElementClickInterceptedException:
            # 스크롤을 조금씩 위로 올리면서 클릭을 시도
            height = -100
            while True:
                driver.execute_script(f"window.scrollBy(0, {height});")
                time.sleep(1)  # 스크롤 후 로딩 대기
                try:
                    more_review_button = driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div/div[6]/div[2]/div[3]/div[2]/div/a')
                    if more_review_button.is_displayed() and more_review_button.is_enabled():
                        more_review_button.click()
                        i += 1
                        time.sleep(2)  # 클릭 후 로딩 대기
                        break  # 클릭 성공 시 루프 탈출
                except ElementClickInterceptedException:
                    height -= 100  # height를 -100씩 더 줄임
                except:
                    break
        except NoSuchElementException:
            break
        except ElementNotInteractableException:
            break
            
            
    # 개별 리뷰 더보기 버튼
    # 초기 XPath
    base_xpath = '/html/body/div[3]/div/div/div/div[6]/div[2]/div[3]/div/ul/li[{}]/div/div[4]/a'

    # 버튼을 순차적으로 클릭
    button_index = 1
    while True:
        try:
            # 현재 XPath
            current_xpath = base_xpath.format(button_index)

            # 해당 XPath에 해당하는 요소가 있는지 확인
            button = driver.find_element(By.XPATH, current_xpath)

            # 요소가 있다면 클릭하고 인덱스를 증가시킴
            button.click()
            time.sleep(2)  # 클릭 후 페이지 로딩 대기
            button_index += 1
        except:
            break
            
    scroll_up()
        
    while True:
        try:
            # CSS 선택자를 사용하여 'sIv5s WPk67' 클래스를 가진 <a> 태그 찾기
            review_buttons = driver.find_elements(By.CSS_SELECTOR, 'a.sIv5s.WPk67[role="button"]')

            if not review_buttons:
                # 더 이상 클릭할 버튼이 없으면 종료
                break

            for button in review_buttons:
                try:
                    button.click()
                    time.sleep(2)  # 클릭 후 페이지 로딩 대기
                except ElementNotInteractableException:
                    continue
        except NoSuchElementException:
            print("리뷰 더보기 버튼을 찾을 수 없습니다.")
            break
        except Exception as e:
            print(f"오류 발생: {e}")
            break
            
    scroll_down()

    # 페이지 로드 후 HTML 소스를 BeautifulSoup로 파싱
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    
    try:
        review_texts = soup.find_all('span', class_='zPfVt')
        keyword_review_texts = soup.find_all('div', class_='ERkm0')
    except:
        review_texts = float('nan')
        keyword_review_texts = float('nan')

    # 텍스트 추출
    review_text_list = [span.get_text() for span in review_texts]
    keyword_review_texts_list = [div.get_text(separator = ',') for div in keyword_review_texts]
    
    return review_keywords_data, review_text_list, keyword_review_texts_list

# 크롤링 실행 (음식점 한 곳 크롤링 완료 시점마다 저장)

In [9]:
import os
import pandas as pd
from selenium import webdriver
from datetime import datetime

# 데이터프레임 로드
dong = pd.read_csv('./서교동.csv', dtype={'ID': str})

# Chrome WebDriver 초기화
driver = webdriver.Chrome()

# driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

# 결과 저장을 위한 빈 리스트 생성
results = []

totalnum = len(dong)
file_path = '서교동_상세정보_from1068.csv'

# 기존 파일이 존재하면 삭제 (중복 방지)
if os.path.exists(file_path):
    os.remove(file_path)

# 각 ID에 대해 데이터를 수집하고 리스트에 저장
for idx, row in dong[1068:].iterrows():
    start_time = datetime.now()  # 시작 시간 기록

    ID = row['ID']
    url = f'https://m.place.naver.com/restaurant/{ID}/home?entry=pll'
    driver.get(url)
    
    home_data = home_page_data()
    menu_data = menu_page_data()
    info_data = info_page_data(ID)
    review_data = review_page_data()
    
    # 데이터를 하나의 튜플로 결합
    combined_data = home_data + (menu_data,) + info_data + review_data
    results.append(combined_data)
    
    # 결과를 데이터프레임으로 변환
    columns = [
        'restaurant_name', 'category', 'second_count_review', 'address', 'distance', 
        'business_hours', 'phone_number', 'website_links', 'home_service', 'menu_contents', 
        'services', 'parking', 'seats', 'info','review_keywords_data', 'review_text_list', 'keyword_review_texts_list'
    ]
    results_df = pd.DataFrame([combined_data], columns=columns)

    # 데이터프레임을 CSV 파일에 추가 (모드 'a'로 열고 헤더는 첫 번째에만 씀)
    header = not os.path.isfile(file_path)
    results_df.to_csv(file_path, mode='a', header=header, index=False, encoding='utf-8-sig')
    
    end_time = datetime.now()  # 종료 시간 기록
    elapsed_time = end_time - start_time  # 걸린 시간 계산
    
    print(totalnum, '중', idx + 1, '완', '현재 시각:', end_time.strftime('%Y-%m-%d %H:%M:%S'), '걸린 시간:', elapsed_time)


1964 중 1069 완 현재 시각: 2024-07-13 20:47:04 걸린 시간: 0:02:43.300078
1964 중 1070 완 현재 시각: 2024-07-13 20:49:30 걸린 시간: 0:02:25.589307
1964 중 1071 완 현재 시각: 2024-07-13 20:54:18 걸린 시간: 0:04:48.359940
1964 중 1072 완 현재 시각: 2024-07-13 20:58:37 걸린 시간: 0:04:18.677358
1964 중 1073 완 현재 시각: 2024-07-13 21:01:01 걸린 시간: 0:02:23.698111
1964 중 1074 완 현재 시각: 2024-07-13 21:04:50 걸린 시간: 0:03:49.098988
1964 중 1075 완 현재 시각: 2024-07-13 21:10:16 걸린 시간: 0:05:26.003175
1964 중 1076 완 현재 시각: 2024-07-13 21:15:06 걸린 시간: 0:04:50.223036
1964 중 1077 완 현재 시각: 2024-07-13 21:20:08 걸린 시간: 0:05:02.447148
1964 중 1078 완 현재 시각: 2024-07-13 21:25:25 걸린 시간: 0:05:16.295526
1964 중 1079 완 현재 시각: 2024-07-13 21:30:55 걸린 시간: 0:05:30.732013
1964 중 1080 완 현재 시각: 2024-07-13 21:35:05 걸린 시간: 0:04:09.403956
1964 중 1081 완 현재 시각: 2024-07-13 21:40:11 걸린 시간: 0:05:06.290736
1964 중 1082 완 현재 시각: 2024-07-13 21:41:46 걸린 시간: 0:01:34.435917
1964 중 1083 완 현재 시각: 2024-07-13 21:46:29 걸린 시간: 0:04:42.925135
1964 중 1084 완 현재 시각: 2024-07-13 21:51:38 걸린 시간: 0:05:09

WebDriverException: Message: disconnected: not connected to DevTools
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF6B801EEA2+31554]
	(No symbol) [0x00007FF6B7F97ED9]
	(No symbol) [0x00007FF6B7E5872A]
	(No symbol) [0x00007FF6B7E3FA9C]
	(No symbol) [0x00007FF6B7E3F960]
	(No symbol) [0x00007FF6B7E5AD51]
	(No symbol) [0x00007FF6B7EEC079]
	(No symbol) [0x00007FF6B7ECCDD3]
	(No symbol) [0x00007FF6B7E9A33B]
	(No symbol) [0x00007FF6B7E9AED1]
	GetHandleVerifier [0x00007FF6B8328B1D+3217341]
	GetHandleVerifier [0x00007FF6B8375AE3+3532675]
	GetHandleVerifier [0x00007FF6B836B0E0+3489152]
	GetHandleVerifier [0x00007FF6B80CE776+750614]
	(No symbol) [0x00007FF6B7FA375F]
	(No symbol) [0x00007FF6B7F9EB14]
	(No symbol) [0x00007FF6B7F9ECA2]
	(No symbol) [0x00007FF6B7F8E16F]
	BaseThreadInitThunk [0x00007FFB50C4257D+29]
	RtlUserThreadStart [0x00007FFB5120AF28+40]


In [1]:
1

1